# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re
import functools

from random import randint

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [3]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Время выполнения функции {func.__name__}: {end - start}')
        return result

    return wrapper

## Задание 2

In [4]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'Функция вызвана с параметрами:\n{args}, {kwargs}')
        return result

    return wrapper

## Задание 3

In [5]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        wrapper.calls += 1
        print(f'Функция была вызвана: {wrapper.calls} раз')
        return result

    wrapper.calls = 0
    return wrapper

## Задание 4

In [6]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}

  @functools.wraps(func)
  def fmemo(*args):
    if args in cache:
      return cache[args]

    result = func(*args)
    cache[args] = result

    return result

  fmemo.cache = cache
  return fmemo

## Тестирование

In [7]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Время выполнения функции word_count: 0.4402931710001212
Функция вызвана с параметрами:
('whole',), {}
Функция была вызвана: 1 раз
Cлово whole встречается 176 раз


In [8]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [9]:
# измеряем время выполнения
start = time.perf_counter()
result = fib(20)
end = time.perf_counter()
print(f'{fib.__name__} потребовалось {end - start:.6f} секунд')

fib потребовалось 0.011709 секунд


In [10]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [11]:
# измеряем время выполнения
start = time.perf_counter()
result = fib(20)
end = time.perf_counter()
print(f'{fib.__name__} потребовалось {end - start:.6f} секунд')

fib потребовалось 0.000155 секунд
